In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix

In [7]:
window_size = 56
n_components = 60

In [8]:
# Cargar las variables independientes (X_train) sin cabecera desde archivo
X_train = np.loadtxt(f'../data/v8/transformed_train_data_comp_{n_components}.csv', delimiter=',')

# Cargar las variables dependientes (y_train) con cabecera desde archivo
y_train = pd.read_csv(f'../data/train_value_min_label_windows_{window_size}_llm.csv')

# Cargar el conjunto de test (X_test sin cabeceras y y_test con cabeceras)
X_test = np.loadtxt(f'../data/v8/transformed_test_data_comp_{n_components}.csv', delimiter=',')
y_test = pd.read_csv(f'../data/test_value_min_label_windows_{window_size}_llm.csv')

# Eliminar la columna "row" que es solo un índice
y_train = y_train.drop(columns=['row'])
y_test = y_test.drop(columns=['row'])


In [9]:
# Escalar los datos de entrenamiento y test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [10]:
from xgboost import XGBClassifier

# Bucle para realizar clasificación binaria para cada columna (clase)
for col in y_train.columns:
    y_train_bin = y_train[col]
    y_test_bin = y_test[col]
    
    # Verificar que haya más de una clase en el conjunto de entrenamiento
    if len(np.unique(y_test_bin)) > 1:
        # Calcular la proporción de clases
        neg_count = np.sum(y_train_bin == 0)
        pos_count = np.sum(y_train_bin == 1)
        
        # Ajustar el peso para desbalanceo de clases
        scale_pos_weight = neg_count / pos_count if pos_count > 0 else 1
        
        # Entrenar modelo XGBoost
        xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
        xgb_clf.fit(X_train_scaled, y_train_bin)
    
        # Predecir en el conjunto de test
        y_pred_bin = xgb_clf.predict(X_test_scaled)
        y_pred_proba = xgb_clf.predict_proba(X_test_scaled)[:, 1]  # Para calcular AUC
    
        # Calcular métricas
        accuracy = accuracy_score(y_test_bin, y_pred_bin)
        f1 = f1_score(y_test_bin, y_pred_bin)
        auc = roc_auc_score(y_test_bin, y_pred_proba)
        report = classification_report(y_test_bin, y_pred_bin)
        cm = confusion_matrix(y_test_bin, y_pred_bin)
    
        # Mostrar resultados
        print(f"Resultados para la clase {col}:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"AUC: {auc:.4f}")
        print("Classification Report:\n", report)
        print(f"Confusion Matrix:\n{cm}")
    else:
        # En caso de tener solo una clase
        accuracy = accuracy_score(y_test_bin, np.full_like(y_test_bin, y_train_bin.iloc[0]))
        
        # Mostrar solo accuracy, el resto no se puede calcular
        print(f"Resultados para la clase {col}:")
        print(f"Accuracy (una sola clase): {accuracy:.4f}")
        print("F1 Score: No calculable (una sola clase)")
        print("AUC: No calculable (una sola clase)")

c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:48:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Resultados para la clase Sockets01:
Accuracy: 0.9957
F1 Score: 0.8414
AUC: 0.9950
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     21060
           1       0.88      0.80      0.84       304

    accuracy                           1.00     21364
   macro avg       0.94      0.90      0.92     21364
weighted avg       1.00      1.00      1.00     21364

Confusion Matrix:
[[21028    32]
 [   60   244]]
Resultados para la clase Sockets02:
Accuracy (una sola clase): 1.0000
F1 Score: No calculable (una sola clase)
AUC: No calculable (una sola clase)
Resultados para la clase Light01:
Accuracy (una sola clase): 1.0000
F1 Score: No calculable (una sola clase)
AUC: No calculable (una sola clase)


c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:48:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Resultados para la clase CE appliance01:
Accuracy: 0.9391
F1 Score: 0.9409
AUC: 0.9452
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94      9726
           1       1.00      0.89      0.94     11638

    accuracy                           0.94     21364
   macro avg       0.94      0.94      0.94     21364
weighted avg       0.95      0.94      0.94     21364

Confusion Matrix:
[[ 9720     6]
 [ 1294 10344]]


c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:48:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

Resultados para la clase Sockets05:
Accuracy: 0.9552
F1 Score: 0.0459
AUC: 0.8213
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98     21260
           1       0.03      0.22      0.05       104

    accuracy                           0.96     21364
   macro avg       0.51      0.59      0.51     21364
weighted avg       0.99      0.96      0.97     21364

Confusion Matrix:
[[20384   876]
 [   81    23]]


c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:48:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Resultados para la clase Washer dryer01:
Accuracy: 0.9997
F1 Score: 0.9821
AUC: 1.0000
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     21172
           1       0.96      1.00      0.98       192

    accuracy                           1.00     21364
   macro avg       0.98      1.00      0.99     21364
weighted avg       1.00      1.00      1.00     21364

Confusion Matrix:
[[21165     7]
 [    0   192]]


c:\Users\DennysMallqui\anaconda3\envs\py311\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:48:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Resultados para la clase Washer dryer02:
Accuracy: 0.9876
F1 Score: 0.7342
AUC: 0.9919
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     20839
           1       0.78      0.70      0.73       525

    accuracy                           0.99     21364
   macro avg       0.88      0.85      0.86     21364
weighted avg       0.99      0.99      0.99     21364

Confusion Matrix:
[[20733   106]
 [  159   366]]
